# Multi-Sensor Fusion

**Learning Objective**: Understand why robots combine multiple sensors for better perception

**Estimated Time**: 12 minutes

This notebook demonstrates how fusing camera and LIDAR measurements produces more accurate and robust perception than either sensor alone.

In [ ]:
# Install dependencies
!pip install numpy==1.24.0 matplotlib==3.7.0

In [ ]:
from typing import Tuple
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
def simulate_distance_measurement_camera(
    true_distance: float,
    num_samples: int = 100
) -> np.ndarray:
    """
    Simulate depth camera measurements (Intel RealSense).
    """
    noise_std = 0.02 * true_distance  # 2% of distance
    measurements = np.random.normal(true_distance, noise_std, num_samples)
    
    # 5% miss rate
    miss_mask = np.random.random(num_samples) < 0.05
    measurements[miss_mask] = np.nan
    
    return measurements

def simulate_distance_measurement_lidar(
    true_distance: float,
    num_samples: int = 100
) -> np.ndarray:
    """
    Simulate LIDAR measurements (Velodyne VLP-16).
    """
    noise_std = 0.03  # ±3cm
    measurements = np.random.normal(true_distance, noise_std, num_samples)
    
    # 1% miss rate (more reliable)
    miss_mask = np.random.random(num_samples) < 0.01
    measurements[miss_mask] = np.nan
    
    return measurements

def weighted_fusion(
    camera_meas: np.ndarray,
    lidar_meas: np.ndarray,
    camera_weight: float = 0.4,
    lidar_weight: float = 0.6
) -> np.ndarray:
    """
    Fuse camera and LIDAR using weighted average.
    """
    fused = np.zeros(len(camera_meas))
    
    for i in range(len(camera_meas)):
        cam_val = camera_meas[i]
        lid_val = lidar_meas[i]
        
        if np.isnan(cam_val) and np.isnan(lid_val):
            fused[i] = np.nan
        elif np.isnan(cam_val):
            fused[i] = lid_val
        elif np.isnan(lid_val):
            fused[i] = cam_val
        else:
            fused[i] = camera_weight * cam_val + lidar_weight * lid_val
    
    return fused

def calculate_sensor_metrics(
    measurements: np.ndarray,
    true_value: float
) -> dict:
    """
    Calculate accuracy metrics.
    """
    valid_meas = measurements[~np.isnan(measurements)]
    
    if len(valid_meas) == 0:
        return {'rmse': float('inf'), 'bias': float('inf'), 'miss_rate': 1.0}
    
    errors = valid_meas - true_value
    return {
        'rmse': np.sqrt(np.mean(errors ** 2)),
        'bias': np.mean(errors),
        'miss_rate': np.sum(np.isnan(measurements)) / len(measurements)
    }

In [ ]:
# Scenario: Robot measuring distance to obstacle
true_distance = 3.5  # meters
num_samples = 100

# Simulate sensors
camera_measurements = simulate_distance_measurement_camera(true_distance, num_samples)
lidar_measurements = simulate_distance_measurement_lidar(true_distance, num_samples)
fused_measurements = weighted_fusion(camera_measurements, lidar_measurements, 0.4, 0.6)

# Calculate metrics
camera_metrics = calculate_sensor_metrics(camera_measurements, true_distance)
lidar_metrics = calculate_sensor_metrics(lidar_measurements, true_distance)
fused_metrics = calculate_sensor_metrics(fused_measurements, true_distance)

print("=" * 60)
print("MULTI-SENSOR FUSION: Why Robots Use Multiple Sensors")
print("=" * 60)
print(f"\nTrue distance: {true_distance:.2f} meters\n")
print("Performance Comparison:")
print(f"\n  Camera (Intel RealSense D455):")
print(f"    • RMSE: {camera_metrics['rmse']:.4f} m")
print(f"    • Bias: {camera_metrics['bias']:.4f} m")
print(f"    • Miss Rate: {camera_metrics['miss_rate']*100:.1f}%")
print(f"\n  LIDAR (Velodyne VLP-16):")
print(f"    • RMSE: {lidar_metrics['rmse']:.4f} m")
print(f"    • Bias: {lidar_metrics['bias']:.4f} m")
print(f"    • Miss Rate: {lidar_metrics['miss_rate']*100:.1f}%")
print(f"\n  Fused Estimate (Weighted Fusion):")
print(f"    • RMSE: {fused_metrics['rmse']:.4f} m")
print(f"    • Bias: {fused_metrics['bias']:.4f} m")
print(f"    • Miss Rate: {fused_metrics['miss_rate']*100:.1f}%")

improvement = ((camera_metrics['rmse'] - fused_metrics['rmse']) / camera_metrics['rmse']) * 100
print("\n" + "=" * 60)
print(f"KEY INSIGHT: Fusion improves accuracy by {improvement:.1f}%")
print("Each sensor has different strengths and weaknesses.")
print("Combining them produces more robust perception!")
print("=" * 60)

In [ ]:
# Visualize sensor fusion
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

samples = np.arange(len(camera_measurements))

# Individual sensors
ax1.scatter(samples, camera_measurements, alpha=0.5, s=30, label='Camera', color='blue', marker='o')
ax1.scatter(samples, lidar_measurements, alpha=0.5, s=30, label='LIDAR', color='red', marker='s')
ax1.axhline(true_distance, color='green', linestyle='--', linewidth=2, label='True Distance', alpha=0.7)
ax1.set_ylabel('Distance (meters)', fontsize=11)
ax1.set_title('Individual Sensor Measurements', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xlim([-2, len(samples)])

# Fused measurements
ax2.scatter(samples, fused_measurements, alpha=0.6, s=30, label='Fused Estimate', color='purple', marker='^')
ax2.axhline(true_distance, color='green', linestyle='--', linewidth=2, label='True Distance', alpha=0.7)

valid_fused = fused_measurements[~np.isnan(fused_measurements)]
if len(valid_fused) > 0:
    std_fused = np.std(valid_fused)
    ax2.fill_between(samples, true_distance - std_fused, true_distance + std_fused,
                     alpha=0.2, color='purple', label='±1 Std Dev')

ax2.set_xlabel('Sample Number', fontsize=11)
ax2.set_ylabel('Distance (meters)', fontsize=11)
ax2.set_title('Fused Sensor Estimate (More Accurate!)', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim([-2, len(samples)])

plt.suptitle('Multi-Sensor Fusion: Better Together', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Expected Output

You should see:
1. **Metrics**: Fused RMSE ~10-20% lower than camera alone
2. **Visualization**: Purple fused estimate tighter around true distance

**Key Takeaway**: Sensor fusion is fundamental to Physical AI. By combining complementary sensors, robots achieve more accurate and robust perception than any single sensor can provide.